In [1]:
from keras.preprocessing import image #it is like cv2 for working on images in keras
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
img=image.load_img("/content/pexels-peng-louis-1643457.jpg",target_size=(256,256))

FileNotFoundError: ignored

In [ ]:
img #it is not numpy array it is in pil format
#type(img)

In [ ]:
datagen= ImageDataGenerator(
    rotation_range=30, #it is the angle of rotation
    shear_range=0.2,  #image distort in this
    zoom_range=0.2, #zoom in out both
    horizontal_flip=True,
    #vertical_flip=True, as cat cannot be vertical
    width_shift_range=0.2,  #moving image side a little
    height_shift_range=0.2  ##moving image above a little
    #fill_mode='nearest' means that when image shifted then empty pixel populated by nearby pixels, it is default
                                                      #constant(blax pixels),flip(mirroring done on those) is also there

)

In [ ]:
img =image.img_to_array(img)#convert to numpy array

In [ ]:
type(img),img.shape

In [ ]:
#we have to give images in batches for augmentation
input_batch=img.reshape(1,256,256,3) #forth dimension tell batchsize

In [ ]:
i=0
for output in datagen.flow(input_batch,batch_size=1,save_to_dir='/content/aug'): #we use flow when we want to process one image
                                    #that is present in numpy fomat directly and when we want to process image from
                                    #directory then use diff. method of ImageDataGenerator
  i=i+1
  if i==10:
    break

In [ ]:
#applying data augmentation to cat dog classifier and seeing the difference before and after

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle datasets download -d whenamancodes/cat-and-dog-finder

 98% 214M/218M [00:01<00:00, 140MB/s]
100% 218M/218M [00:02<00:00, 114MB/s]


In [5]:
#for unzipping

import zipfile
zip_ref = zipfile.ZipFile('/content/cat-and-dog-finder.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [6]:
import cv2
import os
import random
import numpy as np

In [7]:
mydir = r'/content/training_set/training_set'

In [8]:
categories=['cats','dogs']

In [9]:
data=[]
for i in categories:
  folder_path=os.path.join(mydir,i)

  if i=='cats':
    label=0
  else:
    label=1

  for j in os.listdir(folder_path):

    img_path=os.path.join(folder_path,j)
    img=cv2.imread(img_path)
    #img=cv2.resize(img,(150,150))
    try:
      img = cv2.resize(img, (150, 150))
    except:
      break
    data.append([img,label])

In [10]:
data[0][0].shape

(150, 150, 3)

In [11]:
random.shuffle(data)

In [12]:
X=[]
y=[]
for i in data:
  X.append(i[0])
  y.append(i[1])

In [13]:
X=np.array(X)
y=np.array(y)

In [14]:
X.shape,y.shape

((3095, 150, 150, 3), (3095,))

In [15]:
X=X/255

In [16]:
from keras import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Activation,Dropout,BatchNormalization

In [17]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(150,150,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

In [18]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [19]:
history = model.fit(X,y,epochs=5,validation_split=0.1)

Epoch 1/5
88/88 [==============================] - 17s 39ms/step - loss: 1.7079 - accuracy: 0.5641 - val_loss: 4.2246 - val_accuracy: 0.4419
Epoch 2/5
88/88 [==============================] - 3s 31ms/step - loss: 0.7422 - accuracy: 0.6212 - val_loss: 1.4882 - val_accuracy: 0.4484
Epoch 3/5
88/88 [==============================] - 3s 32ms/step - loss: 0.6366 - accuracy: 0.6894 - val_loss: 1.0318 - val_accuracy: 0.4548
Epoch 4/5
88/88 [==============================] - 3s 32ms/step - loss: 0.5526 - accuracy: 0.7318 - val_loss: 1.7919 - val_accuracy: 0.4839
Epoch 5/5
88/88 [==============================] - 3s 31ms/step - loss: 0.4976 - accuracy: 0.7655 - val_loss: 1.0301 - val_accuracy: 0.5742


In [20]:
history.history['accuracy'][4]

0.7655296325683594

In [21]:
from keras.preprocessing.image import ImageDataGenerator

In [22]:
batch_size=16

#augmentation config for training
train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

#augmentation config for testing
#we do augmentatn in training data not manipulating testing data

test_datagen=ImageDataGenerator(rescale=1./255)

In [23]:
#for reading images from directory and indefinitely generate
#batches of augmented image data

train_generator=train_datagen.flow_from_directory(
    '/content/training_set/training_set',
    target_size=(150,150),
    batch_size=batch_size,
    class_mode='binary'
)

#for validation set

validation_generator=test_datagen.flow_from_directory(
    '/content/test_set/test_set',
    target_size=(150,150),
    batch_size=batch_size,
    class_mode='binary'
)

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [24]:
#now creating same model

model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(150,150,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

In [25]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [26]:
#when we do augmentation then use fit_generator instead of fit

model.fit_generator(
    train_generator,
    steps_per_epoch=8005 //batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=2023 //batch_size)
#we dont use original data instead use generator that contain all aug. images also

Epoch 1/5


<ipython-input-26-92570776e890>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


500/500 [==============================] - 62s 119ms/step - loss: 1.3153 - accuracy: 0.5588 - val_loss: 0.8210 - val_accuracy: 0.5749
Epoch 2/5
500/500 [==============================] - 57s 115ms/step - loss: 0.6554 - accuracy: 0.6483 - val_loss: 0.6116 - val_accuracy: 0.6711
Epoch 3/5
500/500 [==============================] - 58s 117ms/step - loss: 0.5758 - accuracy: 0.7219 - val_loss: 0.5032 - val_accuracy: 0.7574
Epoch 4/5
500/500 [==============================] - 58s 116ms/step - loss: 0.5422 - accuracy: 0.7557 - val_loss: 0.5738 - val_accuracy: 0.7366
Epoch 5/5
500/500 [==============================] - 57s 114ms/step - loss: 0.5044 - accuracy: 0.7747 - val_loss: 0.5846 - val_accuracy: 0.7059
